# Movement data analysis

Analyze the movement data collected during the experiment

## Import data and libraries

In [45]:
import geopandas as gpd
import folium
from folium.plugins import HeatMap
from branca.colormap import linear
from geolib import geohash as geolib
import json
import math


In [46]:

# Read the data from the csv
df = gpd.read_file('data/Test_movements.csv')
df

,participant_id,start_time,end_time,start_geohash,end_geohash,distance(m),mean_of_transport,is_power_saving,geometry
0,119928XAX,2023-11-22 15:07:41+00:00,2023-11-22 15:29:35+00:00,u0m468,u0m714,53753,TRAIN,False,None
1,119928XAX,2023-11-22 15:36:51+00:00,2023-11-22 15:49:36+00:00,u0m714,u0m70f,1365,WALKING,False,None
2,219935XE1,2023-11-12 19:26:43+00:00,2023-11-12 19:44:21+00:00,u0qh02,u0mgtj,22038,TRAIN,False,None
3,219935XE1,2023-11-12 19:49:14+00:00,2023-11-12 19:54:59+00:00,u0mgtj,u0mgth,290,WALKING,False,None
4,219935XE1,2023-11-12 20:01:35+00:00,2023-11-12 21:01:40+00:00,u0mgth,u0m714,103394,TRAIN,False,None
...,...,...,...,...,...,...,...,...,...
431,2198211RX,2023-12-05 07:46:58+00:00,2023-12-05 08:01:57+00:00,u0m44w,u0m44y,1342,WALKING,False,None
432,2198211RX,2023-12-04 21:33:35+00:00,2023-12-04 21:36:53+00:00,u0kcvw,u0kcvm,1347,WALKING,False,None
433,2198211RX,2023-12-05 06:13:44+00:00,2023-12-05 06:16:56+00:00,u0kcvw,u0kcvm,1290,WALKING,False,None
434,2198211RX,2023-12-05 08:02:11+00:00,2023-12-05 08:06:00+00:00,u0m44y,u0m44y,186,ON_BICYCLE,False,None


In [47]:
## Get a list of all the unique participants in df, including the number of occurrences of each participant
participants = df['participant_id'].value_counts()
participants

participant_id
2198211RX     145
219935XE1     140
2197410XTX     57
119928XAX      49
119963XR1      27
219613XI1      11
219827XRX       7
Name: count, dtype: int64

## Create a heatmap of the locations visited

In [48]:
## Extract from df a list of all start_geohashes and end_geohashes
geohashes = list(df['start_geohash']) + list(df['end_geohash'])
len(geohashes)

872

In [49]:
# If we want, we can remove some characters from the end of each geohash to reduce the precision
geohashes = [geohash[:-1] for geohash in geohashes]

In [50]:
# Convert the list of geohashes to a dataframe
geohashes_df = gpd.GeoDataFrame(geohashes, columns=['geohash'])

## Convert the list of geohashes to a geoJSON object


In [51]:
## Convert geohashes to a heatmap in geojson format
def geohashes_to_heatmap(df):
    # Get the distinct geohashes and their counts from the dataframe
    geohashes = df['geohash'].value_counts()

    print(geohashes)

    # Get the maximum count of any geohash
    max_count = math.log(geohashes.max())

    # Convert the geohashes to a list of lists, each containing the geohash and its count
    geohashes = [[geohash, count] for geohash, count in zip(geohashes.index, geohashes)]
    
    # Create a color scale for the heatmap
    color_scale = linear.RdYlBu_10.scale(1, max_count)

    # Convert geohashes to features for geoJSON
    features = []

    for geohash in geohashes:
        # Get the bounds of the geohash
        bounds = geolib.bounds(geohash[0])
        color = color_scale(math.log(geohash[1]))

        # Create a feature for the geohash
        features.append({
            "type": "Feature",
            "properties": {
                "id": geohash[0],
                "fillColor": color,
                "fillOpacity": 0.6,
                "stroke": False
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": [[
                    [bounds.sw.lon, bounds.sw.lat],
                    [bounds.sw.lon, bounds.ne.lat],
                    [bounds.ne.lon, bounds.ne.lat],
                    [bounds.ne.lon, bounds.sw.lat],
                    [bounds.sw.lon, bounds.sw.lat]
                ]]
            },
        })

    # Convert the geohashes to a heatmap in geojson format
    return {
        "type" : "FeatureCollection",
        "features": features
    }

In [52]:
# Convert the geohashes to a heatmap in geojson format
heatmap = geohashes_to_heatmap(geohashes_df)

# Save GeoJSON with double quotes
with open('results/heatmap.geojson', 'w') as f:
    json.dump(heatmap, f)

geohash
u0m44    142
u0kcu    116
u0m70    109
u0m71     77
u0m46     55
        ... 
u0m4e      1
u0kbg      1
u0q1e      1
u0q1u      1
u0m6j      1
Name: count, Length: 61, dtype: int64


In [53]:
# Visualize as a heatmap using Folium
# Create a folium map centered at an initial location
m = folium.Map(location=[46.9446011, 7.4143311], zoom_start=5)

# Define a style function to set the color of the polygon
def style_function(feature):
    return {
        'fillColor': feature["properties"]["fillColor"],  # Change this to the desired color
        'stroke': feature["properties"]["stroke"],
        'fillOpacity': feature["properties"]["fillOpacity"],
    }

# Add GeoJSON data to the map with the style function
folium.GeoJson(
    heatmap,
    name='Polygon Layer',
    style_function=style_function,
).add_to(m)

# Add Layer Control to the map
folium.LayerControl().add_to(m)

# Save or display the map
m.save("folium_map_with_colored_polygon.html")
